# Welcome to NLP Labs

## Environment configuration

1. Python
2. NLTK http://www.nltk.org/install.html
3. Jupyter https://jupyter.org/ 
4. Anaconda https://www.anaconda.com/download

use symbols to express math: $$y = x^2$$

In [7]:
import nltk
from nltk import word_tokenize, pos_tag
# maybe you need to download NLTK data
# nltk.download('all')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

In [4]:
tokens = word_tokenize("Hi, welcome to the NLP lab!")

In [3]:
print(pos_tag(tokens))

[('Hi', 'NNP'), (',', ','), ('welcome', 'NN'), ('to', 'TO'), ('the', 'DT'), ('NLP', 'NNP'), ('lab', 'NN'), ('!', '.')]


## Exploring the Penn TreeBank (PTB) Corpus

In [111]:
nltk.download('treebank')
annotated_sent = nltk.corpus.treebank.tagged_sents()
 
print(annotated_sent[0])
print("Tagged sentences: ", len(annotated_sent))
print("Tagged words:", len(nltk.corpus.treebank.tagged_words()))

# tagsets
nltk.download('tagsets')
print(nltk.help.upenn_tagset())

[nltk_data] Downloading package treebank to /home/sdam04/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words: 100676
[nltk_data] Downloading package tagsets to /home/sdam04/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890

## Training

In [113]:
# TODO: improve this feature extraction function
    
def features(sentence, index):
    return {
        'word': sentence[index],
        'is_capitalized': sentence[index][0].upper() == sentence[index][0],
        'prefix-1': sentence[index][0],
        'suffix-1': sentence[index][-1],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1]
    }
import pprint 
pprint.pprint(features(['This', 'is', 'a', 'sentence'], 2))

{'is_capitalized': False,
 'next_word': 'sentence',
 'prefix-1': 'a',
 'prev_word': 'is',
 'suffix-1': 'a',
 'word': 'a'}


In [33]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [106]:
cutoff = int(.75 * len(annotated_sent))
training_sentences = annotated_sent[:cutoff]
test_sentences = annotated_sent[cutoff:]
 
print(len(training_sentences))
print(len(test_sentences))
 
def transform_to_dataset(tagged_sentences):
    X, y = [], []
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(training_sentences)

2935
979


# Implementing a classifier

In [107]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline

size=10000

clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X[:size], y[:size])
 
print('training OK')
 
X_test, y_test = transform_to_dataset(test_sentences)
 
print("Accuracy:", clf.score(X_test, y_test))

training OK
Accuracy: 0.880684557287


# Using the classifier

In [59]:
def pos_tag(sentence):
    print('checking...')
    tagged_sentence = []
    tags = clf.predict([features(sentence, index) for index in range(len(sentence))])
    return zip(sentence, tags)

import platform
print(platform.python_version())

print(list(pos_tag(word_tokenize('Hello world, lets do something awesome today!'))))

3.5.3
checking...
[('Hello', 'DT'), ('world', 'NN'), (',', ','), ('lets', 'VBD'), ('do', 'NN'), ('something', 'VBG'), ('awesome', 'IN'), ('today', 'IN'), ('!', 'CD')]


## Rule-based POS taggers

1. **DefaultTagger** that simply tags everything with the same tag

2. **RegexpTagger** that applies tags according to a set of regular expressions

3. ** N-Gram** (n-gram tagger is a generalization of a unigram tagger whose context is the current word together with the part-of-speech tags of the n-1 preceding token)
    - **UnigramTagger**
    - **BigramTagger**
    - **TrigramTagger**

In [118]:
#nltk.download('brown')

from nltk.corpus import brown
from nltk import DefaultTagger as df
from nltk import UnigramTagger as ut
from nltk import BigramTagger as bt
from nltk import TrigramTagger as tg

size = int(len(brown_tagged_sents) * 0.9)

patterns = [(r'.*ing$', 'VBG'), (r'.*ed$', 'VBD'), (r'.*es$', 'VBZ'), (r'.*ould$', 'MD'), (r'.*\'s$', 'NN$'),               
             (r'.*s$', 'NNS'), (r'^-?[0-9]+(.[0-9]+)?$', 'CD'), (r'.*', 'NN')]

brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')

train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

def_model = nltk.DefaultTagger('NN')
uni_model = nltk.UnigramTagger(train_sents)
bi_model = nltk.BigramTagger(train_sents)
tri_model = nltk.TrigramTagger(train_sents)
regexp_model = nltk.RegexpTagger(patterns)

# performance of Default Tagger
print(def_model.evaluate(train_sents))
print(def_model.evaluate(test_sents))
print()
# performance of Unigram Tagger
print(uni_model.evaluate(train_sents))
print(uni_model.evaluate(test_sents))
print()
# performance of Bigram Tagger
print(bi_model.evaluate(train_sents))
print(bi_model.evaluate(test_sents))
print()
# performance of Trigram Tagger
print(tri_model.evaluate(train_sents))
print(tri_model.evaluate(test_sents))
print()
# performance of Regex Tagger
print(regexp_model.evaluate(train_sents))
print(regexp_model.evaluate(test_sents))
print()

0.131405972094873
0.1262832652247583

0.9353630649241612
0.8124190172430977

0.7875962483843528
0.10276088906608193

0.8241954905491543
0.06279278381341573

0.20310204261994455
0.2047244094488189



## Exercise 01

### In this lab you will learn how to train your own POS tagger classifier and test it against some pre-trained models

#### Pleases implement your code and upload it to git using (jupyter notebook format)

#### Classifiers

1. ``model1`` = your POS tagger model (english)
2. ``model2`` = pre-trained POS tagger model using NLTK (maxentropy english)
3. ``model3.x`` = rule-based classifiers (x = 1 to 5)

3. ``model4`` = your POS tagger model (not english)
4. ``model5`` = pre-trained POS tagger model using RDRPOSTagger [1](http://rdrpostagger.sourceforge.net/) or TreeTagger [2](http://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/) (not english)

note: for ``model1`` and ``model3`` you can try different classifiers: Hidden Markov Model, Logistic Regression, Maximum Entropy Markov Models, Decision Trees, Naive Bayes, etc..**choose one**!

#### Corpora
1. ``X1`` = nltk.corpus.treebank (english)
2. ``X2`` = nltk.corpus.brown (english)
3. ``X3`` = other language (not english)

**note**: data split for training/test = 0.8/0.2 (sequencial)

#### Task 1
- get results for english (**plot a graph with all classifiers x results**)
    - ``performance 1.1`` = ``model1`` in ``X1``
    - ``performance 1.2`` = ``model2`` in ``X1``
    - ``performance 1.3.x`` = ``model3.x`` in ``X1``
    
    - ``performance 1.4`` = ``model1`` in ``X2``
    - ``performance 1.5`` = ``model2`` in ``X2``
    - ``performance 1.6.x`` = ``model3.x`` in ``X2``

#### Task 2
- train your model with standard features (**plot a graph with all classifiers x results**)
    - ``performance 2.1`` = ``model4`` in ``X3``
    - ``performance 2.2`` = ``model5`` in ``X3``

### notes:

#### 1. you can save your trained models using pickle (import pickle)
#### 2. please upload your jupyter file to git
#### 3. this script just gives a general idea, please organize and comment your code accordingly
#### 4. you have to make sure the language you choose is supported for one of the classifiers suggested (see above) AND you are able to find a corpus in that language (example: [Tiger Corpus for German](http://www.ims.uni-stuttgart.de/forschung/ressourcen/korpora/TIGERCorpus/download/start.html)). You can also search the Web in order to try to find a pre-trained classifier in your language. If that is not possible, just choose one existing. Please also make sure the language you have choosen _does not_ overlap with other students.
#### 5. If you are able to find an annotated corpus and format is CoNLL, you can easly read it using the following method in NLTK:
```corp = nltk.corpus.ConllCorpusReader()```
#### 6. a nice library to create charts: https://plot.ly/python/bar-charts/